In [1]:
import tables as tb
import numpy as np

载入我的一个自定义数据库, 详细介绍见: [Bunch 转换为 HDF5 文件：高效存储 Cifar 等数据集](https://www.cnblogs.com/q735613050/p/9244223.html)

In [2]:
h5 = tb.open_file('E:/xdata/X.h5')   

cifar = h5.root.cifar10    # 获取 cifar10 数据集

label_names = np.asanyarray(cifar.label_names, dtype='U')
print(label_names)   # 打印类别名字

print(np.unique(cifar.testY))   # 验证类别数是否符合
print(np.unique(cifar.trainY))

['airplane' 'automobile' 'bird' 'cat' 'deer' 'dog' 'frog' 'horse' 'ship'
 'truck']
[0 1 2 3 4 5 6 7 8 9]
[0 1 2 3 4 5 6 7 8 9]


- `np.unique` 的作用是: 去除重复元素. 符合要求.
- `np.bincount` 对整数数组的元素计数.

In [3]:
print(np.bincount(cifar.testY))
print(np.bincount(cifar.trainY)) 

[1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
[5000 5000 5000 5000 5000 5000 5000 5000 5000 5000]


从结果可以看出, `cifar10` 数据集是均衡的.

In [4]:
X_train = np.asanyarray(cifar.trainX, dtype=np.float).reshape((-1, 32*32*3))
X_test = np.asanyarray(cifar.testX, dtype=np.float).reshape((-1, 32*32*3))
y_train = np.array(cifar.trainY)
y_test= np.array(cifar.testY)

# 数据预处理

## 划分数据集

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

## 标准化处理

In [6]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()  
sc.fit(X_train)   # 计算数据中的每个特征的样本均值和标准差
X_train_std = sc.transform(X_train)   # 使用样本均值和标准差做标准化处理
X_val_std = sc.transform(X_val)

In [7]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l1', C=1.0)
# Note that C=1.0 is the default. You can increase
# or decrease it to make the regulariztion effect
# stronger or weaker, respectively.
lr.fit(X_train_std, y_train)
print('Training accuracy:', lr.score(X_train_std, y_train))
print('Test accuracy:', lr.score(X_val_std, y_val))

Training accuracy: 0.5345142857142857
Test accuracy: 0.3668


## 类标 one-hot 编码

In [ ]:
import pandas as pd

ohe_test_y = pd.get_dummies([label_names[name] for name in cifar.testY])
ohe_train_y = pd.get_dummies([label_names[name] for name in cifar.trainY])